In [1]:
#!pip install jdc
%reset
import jdc
import re
import requests
import time
import nltk
import warnings

import pandas as pd

from bs4 import BeautifulSoup as bs

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options

import yfinance as yf

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


# Probleemomschrijving
Het doel is om een pipeline te creeën die data ophaalt, transformeert en laad om vervolgens gebruikt te kunnen worden om de prijs van de ETF VUSA te voorspellen. Hierbij gaan wij artikelen van investopedia en yahoo finance scrapen en hier sentiment analysis op toepassen. Ook halen wij de historische prijsdata op van de verschillende aandelen die zich in de ETF bevinden. En als laatst halen wij de historische prijsdata op van VUSA zelf op en voegen wij alles samen tot 1 DataFrame. Deze pipeline kan elke dag gerund worden om zo nieuwe artikelen op te halen en deze toe te voegen aan de dataset, zo krijg je een steeds beter bruikbare dataset.

# Domeinonderzoek
Vusa (IE00B3XXRP09) is een ETF die beheert wordt door Vanguard group (Fondsbeheerder|NOT,Disclosed|Vanguard S&P 500 UCITS ETF (EUR)|ISIN:IE00B3XXRP09, n.d.). Een ETF is een aandeel wat eigenlijk bestaat uit meerdere aandelen, obligaties of andere effecten die op de markt verhandelt worden (Wat Zijn ETF’s | Educatie | BlackRock, n.d.). VUSA specifiek volgt de marktindex van de SP500, dat zijn aandelen van de 500 grootste bedrijven van de Verenigde Staten. 

De SP500 geeft een goed beeld van de amerikaanse markt, aangezien het veel verschillende sectoren en aandelen van grote bedrijven binnen deze sectoren bevat. SP500 heeft daarom een unieke ligging in de aandelenmarkt, omdat het zo groot kunnen er duidelijke veranderingen in de amerikaanse markt worden gemeten naar aanleiding van de SP500. Omdat de SP500 zo'n goed beeld geeft van de amerikaanse markt worden veel beleggingen hiermee vergeleken om te kijken hoe goed het gaat. (S&P 500 | Wat Is De S&P 500? | Beleggingswiki - Semmie.nl, n.d.).

# Vereiste informatie, databronnen en formaten ruwe data
Wij willen willen artikelen scrapen om daar sentiment analysis op toe te passen. Wij zijn van mening dat dit belangrijke data is en deze bron kan dat bevestigen: (SWOCC, 2020). De rauwe data komt in de vorm van text voor yahoo finance en een beautifulsoup object voor investopedia.
- https://www.investopedia.com/markets-news-4427704
- https://finance.yahoo.com/

<br>
<br>

Ook willen wij gebruik maken van de prijsgeschiedenis van de aandelen die te vinden zijn in de VUSA ETF, de API geeft deze data als dataframe terug. Wij hebben de API hier gevonden:
- https://github.com/ranaroussi/yfinance

Dit hebben wij vervolgens via pip geinstalleerd en geimporteerd via deze code: "import yfinance as yf" die aan het begin van het bestand wordt gebruikt.

<br>
<br>

Als laatst willen wij de prijsgeschiedenis gebruiken van de VUSA zelf, hier hebben wij een publieke dataset voor gedownload van:
- https://datahub.io/core/s-and-p-500

Echter is de website veranderd en is deze dataset niet meer beschikbaar in deze vorm. Hieronder is een link die laat zien hoe de website er voorheen uitzag:

- https://web.archive.org/web/20230402111356/https://datahub.io/core/s-and-p-500

De data kwam in een csv bestand en aangezien wij dit destijds hadden gedownload gebruiken wij nu het gedownloade bestand in de pipeline.

<br>
<br>

Om de juiste data uit de yfinance API te halen en de artikelen van investopedia te filteren hebben wij gebruik gemaakt van een csv bestand wat wij ook gedownload hebben van datahub, aangezien de gehele website is veranderd is ook deze URL niet meer bruikbaar en gebruiken wij de voorheen gedownloade CSV. De originele URL is:

- https://datahub.io/core/s-and-p-500-companies

En om te zien hoe de website eruit zag kan je deze link volgen:
- https://web.archive.org/web/20230601125536/https://datahub.io/core/s-and-p-500-companies

# Het proces
In deze cell zullen wij uitleggen en beschrijven wat er allemaal in de class Pipeline gebeurt. We zullen de methodes uitleggen en hoe wij de de verschillende nodige technieken hebben toegepast.

## ETL
Wij maken in dit project gebruik van ETL (Extract, transform, load). Hiervoor hebben wij ook een ETL methode aangemaakt onderaan de class, hier worden de Extract, transform en load methodes aangeroepen die vervolgens de stappen uivoeren die nodig zijn.


## Extract
In de Extract methode halen wij alle ruwe data uit de beschikbare bronnen. 

Het begint met het aanroepen van de get_response methode met de URL voor investopedia (https://www.investopedia.com/markets-news-4427704), get_response doet hier een get request en zet de html om in een beautifulsoup object.

vervolgens werden de csv bestanden gedownload van datahub.io maar aangezien dit niet meer beschikbaar is, is dit uigecomment. En in plaats hiervan worden de gedownloade bestanden ingelezen als dataframes en in de variabelen: vusa_df en time_data_df gezet.

Dan wordt de yahoo finance website gescraped doormiddel van de get_yahoo_news_articles_df methode, deze methode weigert de niet-essentiele cookies en scrollt dan door de yahoo_finance website(https://finance.yahoo.com/) om zo alle URLs op te halen van de artikelen die op de website staan. Dit doen we doormiddel van een headless selenium browser, aangezien de website dynamisch geladen word en zonder scrollen niet alle URLs opgehaald kunnen worden. Deze URLs worden in een DataFrame gezet met de titels van de artikelen en css klassen van de artikelen op de website. We halen de artikelen en css klassen ook op om duidelijk te kunnen zien of alles goed is gegaan en problemen makkelijk op te lossen als het niet zou gaan zoals wij verwachten.

Dan hebben we alle URLs van de artikelen maar de artikelen zelf hebben we nog niet, dus dat is het volgende wat wij doen. 
Dit doen we met de methode scrape_articles waar wij de URLs als parameter meegeven. Deze methode klikt eerst op het weigeren van niet-essentiele cookies en laadt vervolgens 1 voor 1 de verschillende URLs. Sommige van de artikelen zijn groter en yahoo finance laadt dan niet het gehele artikel zien, hiervoor moet eerst op een knop gedrukt worden met de class 
'collapse-button'. Wij maken gebruik van een try, except om deze knop in te drukken wat zorgt voor robuustheid zodat alle artikelen goed geladen kunnen worden. vervolgens worden de artikelen opgehaald en schoongemaakt doormiddel van de regex functie die in de methode clean_text gebruikt wordt. scrape_articles geeft ook de rauwe html van de artikelen terug om zo makkelijk te kunnen zien of alles volgens verwachting is verlopen en fouten makkelijk op te lossen.

Als laatst word yfinance API gebruikt om de prijsgeschiedenis te downloaden, dit wordt gedaan in de download_yahoo_data methode. In deze methode wordt ook de locatie van het bestand met de tickers van de bedrijven in de SP500 meegegeven zodat de methode dit kan gebruiken om te weten welke prijsdata gedownload moet worden. In dit bestand staan de tickers van de bedrijven in de SP500 en dit wordt omgezet naar een lijst en gebruikt voor de finance API om te weten wat te downloaden. Dit wordt gedownload en automatisch in een DataFrame gezet. Ook geeft yfinance automatisch aan welke downloads niet gelukt zijn, dit is goed voor de robuustheid aangezien we dan makkelijk kunnen zien waar het wellicht misgaat. Wij hebben een aantal downloads die sowieso mislukken, dit zijn downloads van bedrijven die niet meer verhandelt worden. Een goed voorbeeld hiervan is facebook, facebook wordt niet meer verhandelt aangezien het bedrijf is veranderd naar meta.

Ook worden de symbolen uit vusa_df gefilterd en deze worden in lijst symbols_in_vusa gezet. 


## Transform
Nadat alle ruwe data is geëxtraheerd gaan we de data transformeren zodat het bruikbaar is voor machine learning. We voegen alle data samen, schonen dit op en vervangen lege waardes. 

We beginnen met het beautifulsoup object van de webpagina van investopedia(https://www.investopedia.com/markets-news-4427704). Met de methodes get_card_titles en get_card_urls halen wij de titels en URLs van de artikelen op vanuit de webpagina. De webpagina van investopedia is niet dynamisch geladen dus kunnen wij alle artikelen uit het beautifulsoup object halen. De titels en URLs worden beide opgehaald om makkelijk te kunnen checken of alle artikelen meegenomen zijn al worden er opmerkelijkheden gespot. Deze titels en URLs worden in een dataframe gezet met de naam investopedia_df. Nu we de URLs hebben gebruiken we de functie get_url_content om de artikelen van de website te scrapen doormiddel van beautifulsoup.
De artikelen worden teruggegeven als ruwe artikelen en schone artikelen die schoongemaakt zijn door de clean_text methode.

De ruwe artikelen worden ook gebruikt om de symbolen van de bedrijven te filteren die genoemd worden in het artikel, dit wordt gedaan met de get_symbols methode. Alle artikelen en symbolen worden toegevoegd aan de investopedia_df zodat daar alles instaat van investopedia. vervolgens word de DataFrame gefilterd op de artikelen die symbolen bevat die ook in symbols_in_vusa voorkomt.

Daarna worden de artikelen van yahoo finance die eerder opgehaald zijn bij het extraheren toegevoegd aan de yahoo_news_df. 

Nu hebben we alle data in 4 verschillende DataFrames staan:
- api_df: Hier staat de gedownloade prijsdata van de bedrijven in die voorkomen in de SP500
- investopedia_df: Hier staan de artikelen van investopedia in, gefilterd op of de bedrijven die genoemt zijn in de artikelen voorkomen in de bedrijven van de SP500
- yahoo_news_df: Hier staan de artikelen in van de yahoo finance website
- time_data_df: Hier staat de prijsgeschiedenis van de SP500 zelf in.

Deze 4 Dataframes worden samengevoegd in de combine_data methode, in deze methode wordt de laatste processing gedaan en alles samengevoegd.

De combine_data methode worden eerst de investopedia_df en yahoo_news_df gefilterd en samengevoegt tot 1 text dataframe genaamd text_df. De artikelen uit de text_df worden schoongemaakt, getokenized, gefilterd en gelemmatized in de clean_and_tokenize methode en toegevoegd als nieuwe kolom aan de text_df. Dan wordt doormiddel van de SentimentIntensityAnalyzer de positieve en negatieve sentiment uit de artikelen gehaald en deze worden apart toegevoegd als kolommen in de text_df. Vervolgens moeten we de data zo transformeren dat het allemaal numeriek wordt en klaar om toegevoegd aan de uiteindelijke dataframe. Dit doen we door de basisstatistieken van de positieve en negatieve sentimentscores op te halen en samen te voegen tot een DataFrame van 1 rij. Nu hebben wij een DataFrame met de statistieken van de sentimentscores van vandaag.

Vervolgens voegen wij de statistieken van de sentimentscores van de text toe bij vandaag in de api_df. Deze wordt dan vervolgens gemerged met de time_data op de key met de datums. Zo hebben we dus nu de prijsgeschiedenis van alle bedrijven in de SP500, de prijsgeschiedenis van de SP500 zelf en de statistieken van de sentimentscores van vandaag. Als dit elke dag gerunt word dan worden langzaam steeds meer rijen met statistieken van sentimentscores van alle dagen toegevoegd.

Als laatst worden de missende waardes vervangen met fillna zodat het bruikbaar wordt voor machine learning.

## Load
In de load functie word er voor gezorgd dat de data makkelijk opgehaald en gebruikt kan worden voor machine learning.

Wij doen dit door de DataFrame op te slaan in een CSV bestand wat later makkelijk gemimporteerd en gebruikt kan worden.

In [2]:
DATA_LOCATION = 'Datasets/'

HEADERS = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'en-US,en;q=0.9,nl;q=0.8',
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'}


URLS = ['https://www.investopedia.com/markets-news-4427704', 'https://finance.yahoo.com/', 
        'https://datahub.io', 'https://datahub.io/core/s-and-p-500', 'https://datahub.io/core/s-and-p-500-companies']

# Set up Pipeline class

In [3]:
class Pipeline:
    """
    Pipeline to get data relating to the stock vusa that tracks the SP500. Includes sentiment analysis from articles,
    price history of stocks that are in the ETF and price history of vusa itself. 
    The class follows the Extract, Transform, Load functionality.
    """
    def __init__(self, data_location, headers, URLs):
        self.DATA_LOCATION = data_location
        
        self.HEADERS = headers
        
        
        self.INVESTOPEDIA_URL = URLs[0]
        self.YAHOO_URL = URLs[1]
        
        self.DATAHUB_URL = URLs[2]
        self.SP_DOWNLOAD_URL = URLs[3]
        self.VUSA_DOWNLOAD_URL = URLs[4]
        
        
        self.OPTIONS = self.set_options()
        nltk.download(['punkt', 'wordnet', 'stopwords', 'omw-1.4', 'vader_lexicon'], quiet=True)

In [4]:
%%add_to Pipeline
def set_options(self):
        """
        Set selenium browser options so that it runs headless
        
        Returns 
        -----------
        OPTIONS : Options
            selenium options with headless browser arguments
        """
        OPTIONS = Options()
        OPTIONS.add_argument('--headless')
        OPTIONS.add_argument('--disable-gpu') 
        return OPTIONS

# Helper methods

In [5]:
%%add_to Pipeline
def progress_bar(self, current, total, bar_length=20):
        fraction = current / total

        arrow = int(fraction * bar_length - 1) * '-' + '>'
        padding = int(bar_length - len(arrow)) * ' '

        ending = '\n' if current == total else '\r'

        print(f'Progress: [{arrow}{padding}] {int(fraction*100)}%', end=ending)

# Extracting

In [6]:
%%add_to Pipeline
def get_response(self, URL):
        """
        Does a get request and returns a beautifulsoup object of the webpage.
        
        Parameters
        -----------
        URL : str
            String of the url to get the content from.
            
        Returns
        -----------
        soup : BeautifulSoup
            A Beautifulsoup object that contains the html of the webpage.
        """
        headers = self.HEADERS
        r = requests.get(URL, headers=headers)
        soup = bs(r.content)
        return soup

In [7]:
%%add_to Pipeline
def get_datahub_csv(self, URL, filename):
        """
        Finds download URL on datahub.io, does a get request to that URL 
        and writes the downloaded CSV to the data location specified in self.DATA_LOCATION 
        with the name given in the parameters.
        
        Parameters
        -----------
        URL : str
            String of the url to get the content from.
        filename : str
            The name that should be given to the final downloaded file.
        
        """
        soup = self.get_response(URL)
        download_url = soup.find_all('table')[1].find_all('a')[1]['href']
        download_url = self.DATAHUB_URL + download_url
        with open(self.DATA_LOCATION + filename, 'wb') as f:
            f.write(requests.get(download_url).content)

In [8]:
%%add_to Pipeline
def get_yahoo_news_articles_df(self):
        """
        Uses a selenium browser with the options specified in self.OPTIONS to open the self.YAHOO_URL, scroll down 
        and get all the titles, URLs and CSS classes of the articles and puts them in a pd.DataFrame.
        
        Returns
        -----------
        df : pd.DataFrame
            DataFrame with the titles of articles, URLs of articles and CSS classes of articles from self.YAHOO_URL.
        
        """
        yahoo_news_driver = webdriver.Chrome(options=self.OPTIONS)

        yahoo_news_driver.get(self.YAHOO_URL)
        yahoo_news_driver.find_element(By.CLASS_NAME, 'reject-all').click()
        actions = ActionChains(yahoo_news_driver)
        for i in range(1500):
            actions.scroll_by_amount(0, 500)
        actions.perform()
        actions.reset_actions()

        links = self.get_article_links(yahoo_news_driver)
        df = pd.DataFrame({'title': [link.text for link in links], 
                           'url': [link.get_attribute('href') for link in links], 
                           'classes': [link.get_attribute('class') for link in links]})

        yahoo_news_driver.quit()
        df = df[df['url'] \
                .str.contains('/news/') & df['classes'].str.contains('js-content-viewer')]\
                .drop_duplicates(subset='url', keep='last')\
                .reset_index(drop=True)
        return df

In [9]:
%%add_to Pipeline
def scrape_articles(self, URLs):
        """
        Uses a list of URLs of articles from yahoo finance to scrape the articles from the URLs, 
        clean them and return the cleaned and raw lists
        
        Parameters
        -----------
        URLs : list[str]
            list of strings of URLs of articles.
            
        Returns
        -----------
        articles : list[str]
            A list of articles cleaned to only have words- and whitespace characters.
            
        raw_articles : list[list[str]]
            A list of lists of html from the paragraphs of the scraped articles.
            
        """
        
        articles = []
        raw_articles = []
        yahoo_driver = webdriver.Chrome(options=self.OPTIONS)
        yahoo_driver.get(self.YAHOO_URL)
        yahoo_driver.find_element(By.CLASS_NAME, 'reject-all').click()

        for index, url in enumerate(URLs):
            yahoo_driver.get(url)
            try:
                yahoo_driver.find_element(By.CLASS_NAME, 'collapse-button').click()
            except:
                pass
            article_p_elements = yahoo_driver.find_element(By.CLASS_NAME, 'caas-body').find_elements(By.TAG_NAME, 'p')
            article_by_paragraph = [p.text for p in article_p_elements if p.text!='']

            raw_article = [bs(element.get_attribute('outerHTML'), "html.parser") for element in article_p_elements]
            article = " ".join(article_by_paragraph)
            clean_article = self.clean_text(article)
            articles.append(clean_article)
            raw_articles.append(raw_article)
            self.progress_bar(index+1, len(URLs))


        yahoo_driver.quit()

        return articles, raw_articles

In [10]:
%%add_to Pipeline
def download_yahoo_data(self, file_path):
        """
        Downloadt gegevens van Yahoo Finance voor tickers die zijn vermeld in het opgegeven CSV-bestand.
        Downloads data from yahoo finance of tickers that appear in the file from the given file path

        Parameters
        -----------
        file_path : str
            File path to a csv containing ticker symbols.

        Returns
        -----------
        filtered_data : pd.DataFrame
            Filtered DataFrame with downloaded data.
        """
        df = pd.read_csv(file_path)
        symbol_list = df.iloc[:, 0].tolist()

        download = yf.download(symbol_list, group_by="ticker")
        data = download.copy()

        filtered_data = data.dropna(axis=1, how='all')
        filtered_data.columns = filtered_data.columns.remove_unused_levels()

        remaining_tickers = list(filtered_data.columns.levels[0])
        missing_tickers = list(set(symbol_list) - set(remaining_tickers))

        return filtered_data    

In [11]:
%%add_to Pipeline
def extract(self):
        # get site data
        self.investopedia_soup = self.get_response(self.INVESTOPEDIA_URL)

        # download csv data
        # get_datahub_csv(self.VUSA_DOWNLOAD_URL, 'vusa_holdings.csv')
        # get_datahub_csv(self.SP_DOWNLOAD_URL, 'time_data.csv')
       
        # get csv data
        self.vusa_df = pd.read_csv(self.DATA_LOCATION + 'vusa_holdings.csv')
        self.time_data_df = pd.read_csv(self.DATA_LOCATION + 'time_data.csv')

        # get yahoo articles with titles, urls and css classes
        self.yahoo_news_df = self.get_yahoo_news_articles_df()
    
        # scrape news articles from yahoo article urls
        self.yahoo_news_articles, self.raw_yahoo_news_articles = self.scrape_articles(self.yahoo_news_df['url'])
        
        # get api data
        self.api_df = self.download_yahoo_data('Datasets/vusa_holdings.csv')

# Transforming

In [12]:
%%add_to Pipeline
def get_card_titles(self):
        """
        Get the titles from the a-tags with the CSS class "card"
        
        Returns
        -----------
        titles : list[str]
            titles of the articles from the html
        """
        titles = []
        a_tags = self.investopedia_soup.find_all('a', class_='card')
        for i in range(len(a_tags)):
            titles.append(a_tags[i].span.text)
        return titles

In [13]:
%%add_to Pipeline
def get_card_urls(self):
        """
        Get the URLs from the a-tags with the CSS class "card"
        
        Returns
        -----------
        URLs : list[str]
            URLs of the articles from the html
        """
        URLs = []
        a_tags = self.investopedia_soup.find_all('a', class_='card')
        for i in range(len(a_tags)):
            URLs.append(a_tags[i]['href'])
        return URLs

In [14]:
%%add_to Pipeline
def get_url_content(self, urls):
        articles = []
        raw_articles = []
        for url in urls:
            soup = self.get_response(url)
            paragraphs = soup.find('div', class_='article-content').find_all('p')
            article = ''.join([p.get_text(separator=' ') for p in paragraphs])
            raw_articles.append(paragraphs)
            articles.append(self.clean_text(article))

        return articles, raw_articles

In [15]:
%%add_to Pipeline
def get_symbols(self, raw_articles, href_ref):
        symbol_list = []
        for p_list in raw_articles:
            article_symbols = []
            for p in p_list:
                symbols = [a_tag.text for a_tag in p.find_all('a') if re.search(href_ref, a_tag['href'])]
                if symbols:
                    for symbol in symbols:
                        article_symbols.append(symbol)
            symbol_list.append(article_symbols)
        return symbol_list

In [16]:
%%add_to Pipeline
def clean_text(self, text):
    return re.sub(r'[^\w\s]', '', text)

In [17]:
%%add_to Pipeline
def get_article_links(self, driver):
    links = driver.find_elements(By.TAG_NAME, 'a')
    return links

In [18]:
%%add_to Pipeline
def clean_and_tokenize(self, articles):
        lemmatizer = WordNetLemmatizer()

        tokenized_articles = []
        for article in articles:
            tokenized_article = nltk.word_tokenize(article)
            tokenized_article = [token for token in tokenized_article if token not in stopwords.words('english')]
            tokenized_article = [lemmatizer.lemmatize(token) for token in tokenized_article]

            tokenized_articles.append(tokenized_article)

        return tokenized_articles

In [19]:
%%add_to Pipeline
def combine_data(self, api_df, investopedia_df, yahoo_df, time_data):
        df1 = investopedia_df.drop(['symbols'], axis=1)
        df2 = yahoo_df.drop('classes', axis=1)
        text_df = pd.concat([df1, df2])
        text_df = text_df.reset_index(drop=True)
        
        # tokenize the articles and add them to a DataFrame
        text_df['tokenized_article'] = self.clean_and_tokenize(text_df['article'])
        
        # apply sentiment analysis and add it to the DataFrame
        analyzer = SentimentIntensityAnalyzer()
        text_df['positivity_score'] = [analyzer.polarity_scores(article)['pos'] for article in text_df['article']]
        text_df['negativity_score'] = [analyzer.polarity_scores(article)['neg'] for article in text_df['article']]
        
        # get statistics from positive and negative sentiment analysis
        pos = text_df.describe().T[:1].reset_index(drop=True)
        neg = text_df.describe().T[1:].reset_index(drop=True)
        
        # rename columns in pos and neg so they can be concattinated
        pos.columns = ['pos_' + col for col in pos.columns]
        neg.columns = ['neg_' + col for col in neg.columns]
        
        # concattinate the positve and negative sentiment analysis statistics to a 1 row DataFrame
        pos_neg_row = pd.concat([pos, neg], axis=1)
        
        api_df = api_df.iloc[::-1].reset_index(drop=False, names='Date')
        
        api_df.columns = ['_'.join(col).strip() for col in api_df.columns.values]
        
        combined_df = pd.concat([api_df, pos_neg_row], axis=1)
        
        combined_df['Date_'] = combined_df['Date_'].astype(str)
        time_data['Date'] = time_data['Date'].astype(str)

        final_df = combined_df.merge(time_data, how='left', left_on='Date_', right_on='Date')
        
        return final_df

In [20]:
%%add_to Pipeline
def transform(self):        
        # filter site data
        titles = self.get_card_titles()
        urls = self.get_card_urls()
        
        # transform filtered data to DataFrame(investopedia_df)
        investopedia_df = pd.DataFrame({'url': urls, 'title': titles})
        
        # get articles from urls
        articles, raw_articles = self.get_url_content(investopedia_df['url'])
                
        # get symbols from article
        symbols = self.get_symbols(raw_articles, 'widgetsymbol')
        
        # filter symbols from the vusa_df csv data
        symbols_in_vusa = self.vusa_df['Symbol'].tolist()
        
        # add articles and symbols to DataFrame(investopedia_df)
        investopedia_df['article'] = articles
        investopedia_df['symbols'] = symbols

        # filter DataFrame(investopedia_df) to only have symbols that occur in the symbols_in_vusa
        investopedia_df = investopedia_df[investopedia_df['symbols'].apply(lambda symbols: any(symbol in symbols_in_vusa for symbol in symbols))]
                
        # add scraped articles to DataFrame
        self.yahoo_news_df['article'] = self.yahoo_news_articles

        # combine DataFrames
        self.df = self.combine_data(self.api_df, investopedia_df, self.yahoo_news_df, self.time_data_df)
        self.df = self.df.dropna(axis=0, thresh=int(self.df.shape[1]*0.2))
        self.df = self.df.fillna(0)

# Loading

In [21]:
%%add_to Pipeline
def load(self, filename):
        self.df.to_csv(filename)

# ETL

In [22]:
%%add_to Pipeline
def ETL(self, filename):
    self.extract()
    self.transform()
    self.load(filename)
        

In [23]:
pipeline = Pipeline(DATA_LOCATION, HEADERS, URLS)

In [24]:
pipeline.ETL('Data.csv')

Progress: [------------------->] 100%
[*********************100%%**********************]  505 of 505 completed


27 Failed downloads:
['CTXS', 'FB', 'VIAC', 'INFO', 'FBHS', 'WLTW', 'DRE', 'KSU', 'ANTM', 'FRC', 'BRK.B', 'DISCK', 'FISV', 'DISCA', 'PKI', 'ATVI', 'NLOK', 'PBCT', 'SIVB', 'TWTR', 'ABC', 'NLSN', 'RE', 'BLL', 'CERN', 'XLNX']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 1925-02-24 -> 2024-02-01)')


In [25]:
pipeline.df

,Date_,KMI_Open,KMI_High,KMI_Low,KMI_Close,KMI_Adj Close,KMI_Volume,NRG_Open,NRG_High,NRG_Low,...,Date,SP500,Dividend,Earnings,Consumer Price Index,Long Interest Rate,Real Price,Real Dividend,Real Earnings,PE10
0,2024-02-01,16.920000,17.049999,16.870001,17.045000,17.045000,2315307.0,53.080002,53.509998,52.869999,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2024-01-31,17.219999,17.250000,16.900000,16.920000,16.920000,16870100.0,53.779999,54.119999,52.619999,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2024-01-30,17.049999,17.230000,17.010000,17.209999,17.209999,12446200.0,54.389999,54.639999,53.700001,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2024-01-29,17.440001,17.490000,17.250000,17.370001,17.087002,13665900.0,54.310001,54.689999,53.450001,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2024-01-26,17.330000,17.469999,17.299999,17.440001,17.155861,14261800.0,53.740002,54.270000,53.529999,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11058,1980-03-21,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11059,1980-03-20,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11060,1980-03-19,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11061,1980-03-18,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Referentielijst
- Fondsbeheerder|NOT,Disclosed|Vanguard S&P 500 UCITS ETF (EUR)|ISIN:IE00B3XXRP09. (n.d.). https://www.morningstar.nl/nl/etf/snapshot/snapshot.aspx?id=0P0000YXKB&tab=4&InvestmentType=FE
- Wat zijn ETF’s | Educatie | BlackRock. (n.d.). BlackRock. https://www.blackrock.com/be/individual/nl/educatie/etfs-uitgelegd#Wat-zijn-ETF'
- S&P 500 | Wat is de S&P 500? | Beleggingswiki - Semmie.nl. (n.d.). Semmie. https://semmie.nl/wiki/sp-500/#:~:text=De%20S%26P%20500%20is%20een,ontwikkeling%20van%20de%20financi%C3%ABle%20markt.
- SWOCC. (2020, June 8). De invloed van mediaberichtgeving op beurskoersen - SWOCC. https://www.swocc.nl/kennisbank-item/de-invloed-van-mediaberichtgeving-op-beurskoersen/
